# Devoir 1

In [1]:
import Pkg
Pkg.add("HiGHS")
Pkg.add("StochasticPrograms")
Pkg.add("Distributions")
Pkg.add("Plots")

using HiGHS
using StochasticPrograms
using Distributions
using Random
using Plots

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


## Exercise 3


Following the example seen in class, with $ c=0.15,\, q=0.25,\, r=0.02 $.

In [2]:
@stochastic_model newsvendor begin
    @stage 1 begin
        @decision(newsvendor, x >= 0)
        @objective(newsvendor, Min, 0.15 * x)
    end
    @stage 2 begin
        @uncertain ω
        @recourse(newsvendor, y >= 0)
        @recourse(newsvendor, w >= 0)
        @objective(newsvendor, Min, - 0.25*y - 0.02*w)
        @constraint(newsvendor, y <= ω)
        @constraint(newsvendor, y + w <= x)
    end
end

set_optimizer(newsvendor, HiGHS.Optimizer)
set_silent(newsvendor)

The scenarios are sampled from a normal distribution given by $\omega \sim N(650,80^2)$.

In [3]:
@sampler Sampler = begin
    N::MvNormal

    Sampler(μ, σ) = new(MvNormal(μ, σ))

    @sample Scenario begin
        x = rand(sampler.N)
        return Scenario(ω = x[1])
    end
end

D = Sampler([650], [80])

Scenario sampler

The samples are drawn with the same seed and solved using HiGHS.

In [4]:
objs = Vector{Float64}()
xs = Vector{Float64}()

N = [5, 10, 100, 1000, 10000]
for n in N
    Random.seed!(33)
    instances = instantiate(newsvendor, D, n, optimizer=HiGHS.Optimizer)

    set_silent(instances)
    optimize!(instances)

    push!(objs, objective_value(instances))
    push!(xs, value(all_decision_variables(instances)[1][1]))
end

In [20]:
plot(N, objs, xscale=:log10, markershape=:circle, title="Objective", xlabel="Number of samples", legend=false)
xlims!(1, 100000)

ErrorException: Cannot convert MvNormal{Int64, PDMats.PDiagMat{Int64, Vector{Int64}}, Vector{Int64}} to series data for plotting

In [ ]:
plot(N, xs, xscale=:log10, markershape=:circle, title=raw"$x^{\star}$", xlabel="Number of samples", legend=false)
xlims!(1, 100000)

Both the objective values and the optimal solutions converge to the analytical solution as the number os instances increase. This is expected as the larger the sample, the more representative it is of the original distribution.

## Exercise 4

In [2]:
# parameters
c = [1.5, 1.8]
q = [3., 4.]
r = [1., 1.2]
;

In [3]:
Random.seed!(33)

N = MvNormal([50, 30], [5,2])

n_scenarios = 100
Δ = [rand(N) for _ in 1:n_scenarios];

### (c)

In [ ]:
# model definition
model = Model(HiGHS.Optimizer)

# first stage variables
@variable(model, x[1:2] >= 0)  # x[1] = $x_b$ and x[2] = $x_e$

# second stage variables (for each scenario)
@variable(model, y[1:n_scenarios,1:2] >= 0)
@variable(model, w[1:n_scenarios,1:2] >= 0)

# second stage constraints
@constraint(model, [i=1:n_scenarios,j=1:2], y[i,j] <= Δ[i][j])
@constraint(model, [i=1:n_scenarios,j=1:2], y[i,j] + w[i,j] <= x[j])

#objective
obj = c'*x
for i=1:n_scenarios
    obj += (- q'*y[i,:] - r'*w[i,:]) / n_scenarios
end
@objective(model, Min, obj)

model

In [ ]:
optimize!(model)

In [ ]:
-objective_value(model)

In [ ]:
value.(x)

### (d)

The optimal value found indicates that the optimal strategy for the baker is to always produce a little bit more than the expected demand. In fact, it produces almost the mean value plus a standard deviation.

The result becomes interesting when compared to the solution of the expected value, which would be precisely the average demand.

### (e)

In [ ]:
# model definition
model = Model(HiGHS.Optimizer)

# first stage variables
@variable(model, x[1:2] >= 0, Int)  # x[1] = $x_b$ and x[2] = $x_e$

# second stage variables (for each scenario)
@variable(model, y[1:n_scenarios,1:2] >= 0)
@variable(model, w[1:n_scenarios,1:2] >= 0)

# second stage constraints
@constraint(model, [i=1:n_scenarios,j=1:2], y[i,j] <= Δ[i][j])
@constraint(model, [i=1:n_scenarios,j=1:2], y[i,j] + w[i,j] <= x[j])

#objective
obj = c'*x
for i=1:n_scenarios
    obj += (- q'*y[i,:] - r'*w[i,:]) / n_scenarios
end
@objective(model, Min, obj)

model

In [ ]:
optimize!(model)

In [ ]:
-objective_value(model)

In [ ]:
value.(x)

### (f)

As the L-Shaped method requires a bounded first stage, we assume an upper bound symmetric to the lower bound (0, to ensure non-negativity) with respect to the mean of the demand, that is, $0 \le x_i \le \mu_i$.

In [ ]:
@stochastic_model boulanger begin
    @stage 1 begin
        @decision(boulanger, 0 <= x[i=1:2] <= [50, 30][i])
        @objective(boulanger, Min, c'*x)
    end
    @stage 2 begin
        @uncertain d[1:2]
        @recourse(boulanger, y[1:2] >= 0)
        @recourse(boulanger, w[1:2] >= 0)
        @objective(boulanger, Min, - q'*y - r'*w)
        @constraint(boulanger, [j=1:2], y[j] <= d[j])
        @constraint(boulanger, [j=1:2], y[j] + w[j] <= x[j])
    end
end

In [ ]:
scenarios = [
    @scenario(d[1:2]=Δ[i], probability=1/n_scenarios)
    for i in 1:n_scenarios
]

instance = instantiate(boulanger, scenarios, optimizer=LShaped.Optimizer)

set_optimizer_attribute(instance, MasterOptimizer(), HiGHS.Optimizer)
set_optimizer_attribute(instance, SubProblemOptimizer(), HiGHS.Optimizer)

optimize!(instance)

In [ ]:
objective_value(instance)